In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.applications import EfficientNetB0
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import pyarrow.parquet as pq
import glob


In [3]:
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [25]:
# # Function to load and preprocess data from parquet files
# def load_data(file_paths):
#     data_frames = []
    
#     for file_path in file_paths:
#         print(f"Loading file: {file_path}")
#         # Read parquet file
#         df = pq.read_table(file_path).to_pandas()
#         data_frames.append(df)
    
#     # Concatenate all data frames
#     combined_df = pd.concat(data_frames, ignore_index=True)
#     print(f"Combined dataframe shape: {combined_df.shape}")
    
#     return combined_df


In [21]:
file_paths = glob.glob('*.parquet')
file_paths

['top_gun_opendata_3.parquet']

In [23]:
# file_paths = glob.glob('*.parquet')
    
# if not file_paths:
#     print("No parquet files found. Please provide the dataset files.")
    
# print(f"Found {len(file_paths)} parquet files: {file_paths}")
    
#     # If there are more than 3 files, select a representative subset
# if len(file_paths) > 3:
#     # Select one from beginning, middle, and end for better representation
#     indices = [0, len(file_paths) // 2, len(file_paths) - 1]
#     selected_files = [file_paths[i] for i in indices]
#     print(f"Selected 3 representative files: {selected_files}")
# else:
#     selected_files = file_paths
    
# # Load and preprocess data
# df = load_data(selected_files)
    
# # Display dataset info
# print("\nDataset Information:")
# print(f"Number of samples: {len(df)}")
# print(f"Columns: {df.columns.tolist()}")
    
# # Check for missing values
# missing_values = df.isnull().sum().sum()
# print(f"Total missing values: {missing_values}")

In [24]:
# import pandas as pd
# import glob
# import numpy as np

# # Function to load data efficiently
# def load_data_efficient(file_paths, use_columns=None, max_rows=10000):
#     data_list = []
#     total_rows_loaded = 0
    
#     for file in file_paths:
#         print(f"Loading {file} ...")
        
#         # Load full dataset with only selected columns
#         df = pd.read_parquet(file, columns=use_columns)
#         # Limit rows to prevent memory crash
#         if total_rows_loaded + len(df) > max_rows:
#             remaining_rows = max_rows - total_rows_loaded
#             df = df.iloc[:remaining_rows]
#             data_list.append(df)
#             break  # Stop if max_rows limit reached
        
#         data_list.append(df)
#         total_rows_loaded += len(df)
    
#     df_final = pd.concat(data_list, ignore_index=True)
#     return df_final

# # Get parquet files
# file_paths = glob.glob('*.parquet')

# if not file_paths:
#     print("No parquet files found. Please provide the dataset files.")

# print(f"Found {len(file_paths)} parquet files: {file_paths}")

# # Select 3 files if more than 3 exist
# if len(file_paths) > 3:
#     indices = [0, len(file_paths) // 2, len(file_paths) - 1]
#     selected_files = [file_paths[i] for i in indices]
#     print(f"Selected 3 representative files: {selected_files}")
# else:
#     selected_files = file_paths

# # Specify only needed columns
# use_columns = ["ieta", "iphi", "X_jet_Track_pT", "X_jet_ECAL", "am"]  # Use only required features

# # Load dataset efficiently (Limit to 100,000 rows max)
# df = load_data_efficient(selected_files, use_columns=use_columns, max_rows=100000)

# # Display dataset info
# print("\nDataset Information:")
# print(f"Number of samples: {len(df)}")
# print(f"Columns: {df.columns.tolist()}")

# # Check for missing values
# missing_values = df.isnull().sum().sum()
# print(f"Total missing values: {missing_values}")


In [26]:
def load_data():
    file_path = glob.glob('*.parquet')
    if not file_paths:
        print("ERROR: No file found.")
        return

    print(f"Found {len(file_path)} parquet files: {file_paths}")
    for file in file_path:
        df = pd.read_parquet(file)
        print(f"Shape : {df.shape}")
        print("Columns:", df.columns.tolist())

load_data()

Found 1 parquet files: ['top_gun_opendata_3.parquet']


In [ ]:
def preprocess_data(df):
    # Extract target variable (particle mass)
    y = df['am'].values
    
    # Extract image features (X_jet with the required channels)
    # Assuming X_jet contains the channels in this order: [Track pT, DZ, D0, ECAL]
    # used all channels as mentioned in the requirements (at least ECAL and Track pT)
    
    # First, let's identify the columns related to X_jet
    x_jet_columns = [col for col in df.columns if 'X_jet' in col]
    
    # Print information about the features
    print(f"Number of X_jet columns: {len(x_jet_columns)}")
    print(f"Sample X_jet column names: {x_jet_columns[:5] if len(x_jet_columns) >= 5 else x_jet_columns}")
    
    # Reshape the data based on ieta, iphi dimensions and channels
    # Assuming the data structure follows the 125x125 matrix with ieta, iphi coordinates
    
    # Get unique ieta and iphi values to determine dimensions
    if 'ieta' in df.columns and 'iphi' in df.columns:
        ieta_dim = len(df['ieta'].unique())
        iphi_dim = len(df['iphi'].unique())
        print(f"Detected dimensions: ieta={ieta_dim}, iphi={iphi_dim}")
    else:
        # Default to the mentioned 125x125 if coordinates are not explicitly in columns
        ieta_dim, iphi_dim = 125, 125
        print(f"Using default dimensions: ieta={ieta_dim}, iphi={iphi_dim}")
    
    # Extract X_jet data and reshape
    # This part might need adjustment based on the actual structure of your data
    try:
        # Approach 1: If X_jet is stored as separate columns for each channel and position
        if len(x_jet_columns) >= ieta_dim * iphi_dim * 4:  # 4 channels mentioned
            X = np.zeros((len(df), ieta_dim, iphi_dim, 4))
            
            # Logic to reshape data from flat columns to 4D tensor
            # This is a placeholder and needs to be adapted to your specific data format
            print("Reshaping data from columns to 4D tensor...")
            
        # Approach 2: If X_jet is already stored as a 4D tensor or can be easily reshaped
        elif 'X_jet' in df.columns and isinstance(df['X_jet'].iloc[0], (np.ndarray, list)):
            X = np.stack(df['X_jet'].values)
            print(f"Loaded X_jet as a tensor with shape: {X.shape}")
            
        else:
            # Another approach: If data is stored differently, adapt accordingly
            print("Data format not recognized. Please adapt the preprocessing code.")
            # Placeholder for demonstration
            X = np.random.rand(len(df), ieta_dim, iphi_dim, 4)
    
    except Exception as e:
        print(f"Error during data reshaping: {e}")
        print("Creating a placeholder tensor for demonstration purposes.")
        X = np.random.rand(len(df), ieta_dim, iphi_dim, 4)
    
    # Normalize the data
    # For each channel separately
    for i in range(X.shape[3]):
        channel_data = X[:, :, :, i]
        mean = np.mean(channel_data)
        std = np.std(channel_data)
        X[:, :, :, i] = (channel_data - mean) / (std + 1e-10)  # Add small epsilon to avoid division by zero
    
    print(f"Processed data shape: X={X.shape}, y={y.shape}")
    return X, y


In [ ]:
def build_cnn_model(input_shape, use_efficient_net=True):
    inputs = tf.keras.Input(shape=input_shape)
    
    if use_efficient_net:
        # Use EfficientNetB0 as base model (without top layers)
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
        
        # Make base model trainable
        base_model.trainable = True
        
        # If input has 4 channels but EfficientNet expects 3, adapt accordingly
        if input_shape[2] != 3:
            # Add a 1x1 convolution to adapt the number of channels
            x = layers.Conv2D(3, (1, 1), padding='same')(inputs)
            x = base_model(x)
        else:
            x = base_model(inputs)
        
        # Add pooling
        x = layers.GlobalAveragePooling2D()(x)
        
    else:
        # Custom CNN architecture
        x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D((2, 2))(x)
        
        x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D((2, 2))(x)
        
        x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D((2, 2))(x)
        
        x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.GlobalAveragePooling2D()(x)
    
    # Regression head
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    
    # Output layer (no activation for regression)
    outputs = layers.Dense(1)(x)
    
    # Create the model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    return model


In [5]:
def compile_and_train(model, X_train, y_train, X_val, y_val, batch_size=32, epochs=100):
    # Compile model
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae']
    )
    
    # Define callbacks
    early_stopping = callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        min_delta=0.0001
    )
    
    reduce_lr = callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-6
    )
    
    # Train model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=batch_size,
        epochs=epochs,
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )
    
    return model, history

In [6]:
def evaluate_model(model, X_test, y_test):
    # Predict on test set
    y_pred = model.predict(X_test).flatten()
    
    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Test Mean Absolute Error: {mae:.4f}")
    print(f"Test R² Score: {r2:.4f}")
    
    # Plot predictions vs actual
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--')
    plt.xlabel('Actual Mass')
    plt.ylabel('Predicted Mass')
    plt.title('Predicted vs Actual Particle Mass')
    plt.savefig('pred_vs_actual.png')
    
    return mae, r2, y_pred

In [ ]:
def main():
    # Find all parquet files in the current directory
    file_paths = glob.glob('*.parquet')
    
    if not file_paths:
        print("No parquet files found. Please provide the dataset files.")
        return
    
    print(f"Found {len(file_paths)} parquet files: {file_paths}")
    
    # If there are more than 3 files, select a representative subset
    if len(file_paths) > 3:
        # Select one from beginning, middle, and end for better representation
        indices = [0, len(file_paths) // 2, len(file_paths) - 1]
        selected_files = [file_paths[i] for i in indices]
        print(f"Selected 3 representative files: {selected_files}")
    else:
        selected_files = file_paths
    
    # Load and preprocess data
    df = load_data(selected_files)
    
    # Display dataset info
    print("\nDataset Information:")
    print(f"Number of samples: {len(df)}")
    print(f"Columns: {df.columns.tolist()}")
    
    # Check for missing values
    missing_values = df.isnull().sum().sum()
    print(f"Total missing values: {missing_values}")
    
    # Extract features and target
    X, y = preprocess_data(df)
    
    # Split the data into train and test sets (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Split train set into training and validation (80% of 80% = 64%, 20% of 80% = 16%)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    
    print(f"\nData split sizes:")
    print(f"Training: {X_train.shape[0]} samples ({X_train.shape[0]/len(df):.1%})")
    print(f"Validation: {X_val.shape[0]} samples ({X_val.shape[0]/len(df):.1%})")
    print(f"Test: {X_test.shape[0]} samples ({X_test.shape[0]/len(df):.1%})")
    
    # Build model
    input_shape = X_train.shape[1:]
    print(f"\nInput shape: {input_shape}")
    
    # Decide which model to use
    use_efficient_net = True  # Set to False to use custom CNN
    model = build_cnn_model(input_shape, use_efficient_net=use_efficient_net)
    
    # Display model summary
    model.summary()
    
    # Train model
    model, history = compile_and_train(model, X_train, y_train, X_val, y_val)
    
    # Plot training history
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss (MSE)')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'])
    plt.plot(history.history['val_mae'])
    plt.title('Model MAE')
    plt.ylabel('MAE')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    
    # Evaluate model
    mae, r2, y_pred = evaluate_model(model, X_test, y_test)
    
    # Save the model
    model.save('particle_mass_regression_model.h5')
    print("Model saved as 'particle_mass_regression_model.h5'")
    
    # Return evaluation metrics
    return {
        'mae': mae,
        'r2': r2,
        'model': model
    }

if __name__ == "__main__":
    main()
